In [8]:
import advent
import re

def parse_line(line):
    words = line.split(' ')
    valve = words[1]
    flow = int(words[4].split('=')[1][:-1])
    outs = re.split('valves? ', line)[1].split(', ')
    return (valve, flow, outs)

data = advent.get_lines(16, map_fn=parse_line)

data_dict = dict((line[0], (line[1], line[2])) for line in data)

In [9]:
all_open_valves = '-'.join(sorted([v for v in data_dict if data_dict[v][0] > 0]))
all_open_valves

'CU-FF-GG-IZ-JH-OI-PA-QZ-SZ-TR-TU-UZ-XF-YL-ZL'

In [10]:
# Approach: recurse on state being: (current_valve, opened_valves, minute)
from functools import lru_cache

pressure = lambda valves: sum(map(lambda x: data_dict[x][0], filter(None, valves.split('-'))))
full_pressure = pressure(all_open_valves)

@lru_cache(maxsize=None)
def maximum_pressure(current_valve='AA', opened_valves="", minute=1):
    if minute == 30: return pressure(opened_valves)
    if opened_valves == all_open_valves: return (30 + 1 - minute) * full_pressure

    values = []
    # possible actions: open if current valve is not open
    if current_valve not in opened_valves and current_valve in all_open_valves:
        opened_valves_arr = opened_valves.split('-')
        new_opened_valves = '-'.join(sorted(opened_valves_arr + [current_valve]))
        values.append(maximum_pressure(current_valve, new_opened_valves, minute+1))
    # possible action: move
    # !!! This is dumb. to reduce runtime, only allow moves 'towards' an open valve
    for adj in data_dict[current_valve][1]:
        #print(current_valve, adj, len(opened_valves))
        values.append(maximum_pressure(adj, opened_valves, minute+1))
    
    return pressure(opened_valves) + max(values)

maximum_pressure('AA', "", 1)

1641

In [28]:
# (valve, elephant, openened) -> (minute, max_pressure)
# We override the value if we find a result with a lower minute/higher max_pressure
# becauase with a lower minute, max_pressure will ALWAYS be higher
cache = {}

#@lru_cache(maxsize=None)
def maximum_pressure_elephant(current_valve='AA', current_elephant='AA', opened_valves="", minute=1):
    if minute == 26: return pressure(opened_valves)
    if opened_valves == '-' + all_open_valves: return (26 + 1 - minute) * full_pressure

    # Since human and elephant are symmetric, order then such that human<= elephant
    if current_valve > current_elephant:
        current_valve, current_elephant = current_elephant, current_valve

    # First, lookup in cache:
    # TODO: also look up if there is an entry with a strictly larger opened_valves
    if (current_valve, current_elephant, opened_valves) in cache:
        value = cache[(current_valve, current_elephant, opened_valves)]
        if value[0] < minute: return -999 # This path is not optimal, prune it
        elif value[0] == minute: return value[1] # Cache
        # Otherwise, this path is 'maybe optimal', so continue and store the result in cache

    # Possible actions: open/move. move/open. open/open. or move/move.

    values = []
    # possible actions: open if current valve is not open
    if current_valve not in opened_valves and current_valve in all_open_valves:
        opened_valves_arr = opened_valves.split('-')
        new_opened_valves = '-'.join(sorted(opened_valves_arr + [current_valve]))

        # now check the elephant:
        if current_elephant not in new_opened_valves and current_elephant in all_open_valves:
            opened_valves_arr = new_opened_valves.split('-')
            both_opened_valves = '-'.join(sorted(opened_valves_arr + [current_elephant]))
            values.append(maximum_pressure_elephant(current_valve, current_elephant, both_opened_valves, minute+1))
        # elephant moves
        for adj in data_dict[current_elephant][1]:
            values.append(maximum_pressure_elephant(current_valve, adj, new_opened_valves, minute+1))

    # possible action: move
    for adj in data_dict[current_valve][1]:
        # now check the elephant:
        if current_elephant not in opened_valves and current_elephant in all_open_valves:
            opened_valves_arr = opened_valves.split('-')
            new_opened_valves = '-'.join(sorted(opened_valves_arr + [current_elephant]))
            values.append(maximum_pressure_elephant(adj, current_elephant, new_opened_valves, minute+1))
        # elephant moves
        for adj_elephant in data_dict[current_elephant][1]:
            values.append(maximum_pressure_elephant(adj, adj_elephant, opened_valves, minute+1))

    # Store in cache. We know because of the check earlier, that we can safely override the current value
    best_value = pressure(opened_valves) + max(values)
    cache[(current_valve, current_elephant, opened_valves)] = (minute, best_value)

    return best_value

maximum_pressure_elephant("AA", "AA", "", 1)

2261

In [24]:
# input timestamp: 11  ->10  ->9   ->8  ->7->6  ->5
# seconds taken:   17->41.1->70.6->127->140->205->290
# Projected for timestamp=1: 20 minutes?